## Download and Install necessary libraries

In [ ]:
!conda install beautifulsoup4
!conda install lxml
!conda install html5lib
!conda install requests

from bs4 import BeautifulSoup
import requests
import pandas as pd

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                          --> 4.9.0-py36_0     
    certifi:         2020.4.5.1-py36h9f0ad1d_0 conda-forge --> 2020.4.5.1-py36_0
    openssl:         1.1.1g-h516909a_0         conda-forge --> 1.1.1g-h7b6447c_0

The following packages will be DOWNGRADED:

    ca-certificates: 2020.4.5.1-hecc5488_0     conda-forge --> 2020.1.1-0       


beautifulsoup4-4.9.0 | 165 KB    | ##

## Scrape the table of postal code in toronto from wikipedia page

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [ ]:

raw_table = pd.read_html(source, header=0, attrs={"class":"wikitable sortable"})[0]
raw_table

## Renaming columns

In [ ]:
raw_table = raw_table.rename(columns={"Postcode":"PostalCode", "Borough":"Borough", "Neighbourhood":"Neighborhood"})

## Ignore cells with a borough that is Not assigned

In [ ]:
raw_table = raw_table.drop(raw_table.index[raw_table['Borough']=='Not assigned'])
raw_table

## Remove more than one neighborhood in one postal code area and format it

In [ ]:
table=raw_table.groupby('PostalCode', sort=False, as_index=False).agg(lambda x: ', '.join(set(x)))
table

In [ ]:
for i in range(table.shape[0]):
    if table.loc[i,'Neighborhood'] =='Not assigned':
        table.loc[i,'Neighborhood'] = table.loc[i,'Borough']
table

In [ ]:
table.shape

## Create the geocoder dataframe

In [ ]:
!conda install -c conda-forge geocoder --yes

In [ ]:
geo_coord_table=pd.read_csv("https://cocl.us/Geospatial_data")
geo_coord_table.head()

In [ ]:
print('The table dataframe has {} dimension and geo_coord_table dataframe also has the same {} .'.format(table.shape, geo_coord_table.shape)
     )

### Conform heading of both dataframe

In [ ]:
geo_coord_table.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
geo_coord_table.head()

In [ ]:
geo_coord_table.sort_values(by = ['PostalCode'], inplace=True)
table.sort_values(by = ['PostalCode'], inplace=True)

In [ ]:
Toronto_df = table.merge(geo_coord_table, left_on='PostalCode', right_on='PostalCode')
Toronto_df.head(20)

In [ ]:
Toronto_df.shape

In [ ]:
print('The Toronto_df dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_df['Borough'].unique()),
        Toronto_df.shape[0])
     )

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

In [ ]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
print(Toronto_df['Borough'].unique())

In [ ]:

qp_data = Toronto_df[Toronto_df['Borough'] == "Queen's Park"].reset_index(drop=True)
qp_data.head()

In [ ]:

north_y_data = Toronto_df[Toronto_df['Borough'] == "North York"].reset_index(drop=True)
north_y_data.head()

In [ ]:
north_y_data.shape

In [ ]:
address = 'North York, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Manhattan using latitude and longitude values
map_north_y = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_y_data['Latitude'], north_y_data['Longitude'], north_y_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_y)  
    
map_north_y

In [ ]:

#Get the neighborhood's name.
north_y_data.loc[12, 'Neighborhood']

In [ ]:
neighborhood_latitude = north_y_data.loc[12, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_y_data.loc[12, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_y_data.loc[12, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
results

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

## Explore Neighborhoods in North York  
### Let's create a function to repeat the same process to all the neighborhoods in North York

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
north_y_venues = getNearbyVenues(names=north_y_data['Neighborhood'],
                                   latitudes=north_y_data['Latitude'],
                                   longitudes=north_y_data['Longitude']
                                  )

In [ ]:

north_y_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(north_y_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
north_y_onehot = pd.get_dummies(north_y_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_y_onehot['Neighborhood'] = north_y_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_y_onehot.columns[-1]] + list(north_y_onehot.columns[:-1])
north_y_onehot = north_y_onehot[fixed_columns]

north_y_onehot.head()

In [ ]:
north_y_onehot.shape

In [ ]:

north_y_grouped = north_y_onehot.groupby('Neighborhood').mean().reset_index()
north_y_grouped

In [ ]:

north_y_grouped.shape

In [ ]:

num_top_venues = 5

for hood in north_y_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_y_grouped[north_y_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_y_grouped['Neighborhood']

for ind in np.arange(north_y_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_y_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:

# set number of clusters
kclusters = 5

north_y_grouped_clustering = north_y_grouped.drop('Neighborhood', 1)

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_y_grouped_clustering)
kmeans = KMeans(init = "k-means++", n_clusters = kclusters, n_init = 10).fit(north_y_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_y_merged = north_y_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_y_merged = north_y_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how = 'inner')

north_y_merged # check the last columns!

In [ ]:
north_y_merged.loc[north_y_merged['Cluster Labels'] == 0, north_y_merged.columns[[1] + list(range(5, north_y_merged.shape[1]))]]

In [ ]:

north_y_merged.loc[north_y_merged['Cluster Labels'] == 1, north_y_merged.columns[[1] + list(range(5, north_y_merged.shape[1]))]]

In [ ]:
north_y_merged.loc[north_y_merged['Cluster Labels'] == 2, north_y_merged.columns[[1] + list(range(5, north_y_merged.shape[1]))]]

In [ ]:
north_y_merged.loc[north_y_merged['Cluster Labels'] == 3, north_y_merged.columns[[1] + list(range(5, north_y_merged.shape[1]))]]

In [ ]:
north_y_merged.loc[north_y_merged['Cluster Labels'] == 4, north_y_merged.columns[[1] + list(range(5, north_y_merged.shape[1]))]]